This is a rudimentary version of the signal processing algorithm

In [1]:
import mne
import yasa
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style='white', font_scale=1.2)
from scipy.signal import welch

We will need some way of reading in the data, and splitting it by 30 s epochs.

In [14]:
#Read-in raw data
#filename='sub-02_mne_raw.fif'
#raw= mne.io.read_raw_fif(filename, preload=True, verbose=0)
#Note: this will not work in real-time- I'm unsure how that will work
filename='SC4001E0-PSG.edf'
raw= mne.io.read_raw_edf(filename, preload=True, verbose=0)
times= raw.times
tmax= times.size #Probably isn't in seconds
epoch_num=tmax//30
#for epoch in epoch_num:
    #raw.save('epoch'+epoch+'.fif', tmin=30*(epoch-1), tmax=30*epoch)
#Issue: this will save a separate file for each epoch, which is not efficient and we should avoid
print(epoch_num)
#raw.save('epoch1.fif', tmin=0, tmax=30, overwrite=True) #change to edf?
epoch1_raw=raw.crop(tmax=30) # Also needs to be fixed for correct time units
#Some variable may be needed to keep track of time
#At t=30, the raw data will be split off into its own array and a new array will be created for the next epoch
    #This will probably involve the use of an iterator variable
    #Maybe use 2D array?
#Filter the signal!!

265000
['EEG Fpz-Cz']


In [20]:
#Calculate bandpower or FFT power
#epoch1_raw=mne.io.read_raw_fif('epoch1.fif', preload=True, verbose=0)
#epoch1=epoch1_raw.pick_types(eeg=True)#This doesn't seem to be removing the non-EEG channels
epoch1=epoch1_raw.pick_channels(['EEG Fpz-Cz'])
data=epoch1._data*1e6 #Changing units from V to microvolts
sf = epoch1.info['sfreq']
bp=yasa.bandpower(data, sf=sf) #channel names haven't been set- should be fine since we only have one channel
#print(bp)
alpha=bp.at['CHAN000', 'Alpha']
#print(alpha)
beta=bp.at['CHAN000', 'Beta']
#print(beta)
theta=bp.at['CHAN000', 'Theta']
#Check if it works when 0 waves are detected
#Slow wave detection
slow=yasa.sw_detect(data, sf=sf)
#Check the data types- can get errors if it is not microvolts
#print(slow.summary(grp_chan=True,aggfunc='mean'))
if slow==None:
    slow_num=0
    print(slow_num)
else:
    slow_num=(slow.summary(grp_chan=True,aggfunc='mean')).at['CHAN000', 'Count'] #May need to add paramters for this function
    #print(slow_num)
spindles=yasa.spindles_detect(data, sf=sf)
#print(spindles)
if spindles== None:
    spindles_num=0
    print(spindles_num)
else:
    spindles_shape=(spindles.summary()).size()
    spindles_num=spindles_shape(1) #In theory, spindles_shape will be the dimensions, and this will grab the number of rows
    print(spindles_num)
    #This side of the loop still needs to be tested

05-May-21 16:30:36 | WARNING | No SW were found in channel CHAN000.
05-May-21 16:30:36 | WARNING | No SW were found in data. Returning None.
05-May-21 16:30:36 | WARNING | No spindle were found in channel CHAN000.
05-May-21 16:30:36 | WARNING | No spindles were found in data. Returning None.


0
0


In [39]:
slow_threshold=1 #Placeholder value
spin_threshold=1
#stages=np.zeros(5, dtype='S5') #Check code properly later- this will need to set "stages" to have the same number of elements as epohcs
stages=[] #Try to pre-allocate space if possible
# One possible solution- create an array of maximum possible size, delete all empty elements once staging is done

i=0
#Wakefulness detection

if (alpha+beta>0.5):
    if theta>0.25:
        stages[i]='R'
    else:
        stages[i]='W'
else:
    if slow_num>slow_threshold: #Will need to set a threshold for # of slow waves in N3
        stages[i]='N3'
    elif spindles_num>spin_threshold:
        stages[i]='N2'
    else:
        #stages[i]='N1'
        stages.append('N1')

print(stages)
blank=np.zeros(5, dtype='S5')
print(blank[0])
#Issues: detection doesn't seem to be correct, formatting on array- this will probably work better with integers

['N1']
b''
